# NiN
---
NIN（Network in Network，网络中的网络）是由Min Lin等人在2014年提出的一种卷积神经网络架构。NIN的主要创新在于引入了1x1卷积层和全局平均池化层，这些创新对后续的深度学习模型设计产生了深远的影响。

## 主要特点
1. 1x1卷积层：

    - NIN引入了1x1卷积层，这种卷积层可以在不改变特征图空间维度的情况下，对特征图进行通道间的线性组合。

    - 1x1卷积层可以看作是一种“通道级”的全连接层，能够学习到通道间的非线性关系。

2. 全局平均池化层：

    - NIN在网络的最后使用全局平均池化层（Global Average Pooling, GAP），而不是传统的全连接层。

    - 全局平均池化层将每个特征图的平均值作为输出，这样可以直接得到每个类别的预测分数，减少了参数数量，避免了过拟合。

3. 多层感知器卷积层（MLPConv）：

    - NIN提出了多层感知器卷积层（MLPConv），它由多个1x1卷积层堆叠而成，类似于多层感知器（MLP）。

    - MLPConv能够学习到更复杂的特征表示，提高了网络的表达能力。

## 网络结构
NIN的网络结构相对简单，主要由多个MLPConv块和全局平均池化层组成。以下是一个典型的NIN结构示例：

1. 输入层：

    - 输入图像的尺寸为224x224x3。

2. 第一个MLPConv块：

    - 1个3x3卷积层（步幅1，填充1）

    - 2个1x1卷积层

    - 1个最大池化层（2x2池化窗口，步幅2）

3. 第二个MLPConv块：

    - 1个3x3卷积层（步幅1，填充1）

    - 2个1x1卷积层

    - 1个最大池化层（2x2池化窗口，步幅2）

4. 第三个MLPConv块：

    - 1个3x3卷积层（步幅1，填充1）

    - 2个1x1卷积层

    - 1个最大池化层（2x2池化窗口，步幅2）

5. 全局平均池化层：

    - 对最后一个MLPConv块的输出进行全局平均池化，得到每个类别的预测分数。

6. 输出层：

    - 使用softmax函数进行分类。

![NiN](https://zh-v2.d2l.ai/_images/nin.svg "NiN")

## 优点
    *参数数量少：*通过使用1x1卷积层和全局平均池化层，NIN减少了全连接层的参数数量，避免了过拟合。

    *表达能力强：*MLPConv能够学习到更复杂的特征表示，提高了网络的表达能力。

    *计算效率高：*全局平均池化层减少了计算量，提高了计算效率。

## 缺点
    *训练难度大：*由于网络结构较为复杂，训练难度相对较大。

    *内存消耗大：*尽管减少了全连接层的参数数量，但由于使用了多个1x1卷积层，内存消耗仍然较大。


## 简单代码实现

In [2]:
import torch
from torch import nn
from d2l import torch as d2l


def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU())

最初的NiN网络是在AlexNet后不久提出的，显然从中得到了一些启示。NiN和AlexNet之间的一个显著区别是NiN完全取消了全连接层。 相反，NiN使用一个NiN块，其输出通道数等于标签类别的数量。最后放一个全局平均汇聚层（global average pooling layer），生成一个对数几率 （logits）。NiN设计的一个优点是，它显著减少了模型所需参数的数量。然而，在实践中，这种设计有时会增加训练模型的时间。

In [3]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten())

我们创建一个数据样本来查看每个块的输出形状。

In [4]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])
